In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import json

sns.set_style('darkgrid')

import folium
from folium.plugins import HeatMap
from folium import plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from scipy import stats

In [2]:
# Load in data
prop_df = pd.read_hdf('../data/processed/bexar_processed.h5')

In [3]:
# Pull out addresses and zip codes for criminally-linked properties
mapping_crim_df = prop_df[prop_df['crim_address'].notna()==True][['crim_address','property_zip']].reset_index()

In [7]:
mapping_crim_df.head()

,index,crim_address,property_zip
0,82774,"25319 BOERNE STAGE RD SAN ANTONIO, TX 78255",78255
1,87260,"24424 BLANCO RD SAN ANTONIO, TX 78260",78260
2,94234,"2227 SAWGRASS RDG SAN ANTONIO, TX 78260",78260
3,94265,"2243 WINDING VW SAN ANTONIO, TX 78260",78260
4,94696,"1115 LINKS CV SAN ANTONIO, TX 78260",78260


In [8]:
geolocator = Nominatim(user_agent="mapping_project")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=4)

skip = 0
if skip == 0:
    x = mapping_crim_df['crim_address'].tolist()
    names = []

    for item in x:
        d={}
        a = geocode(item, exactly_one=True, timeout=60)
        try:
            d["Latitude"] = a.latitude
        except:
            pass
        try:
            d["Longitude"] = a.longitude
        except:
            pass
    #     time.sleep(2)
        names.append(d)
        
    mapping_crim_df['coord'] = names

In [9]:
mapping_crim_df.head()

,index,crim_address,property_zip,coord
0,82774,"25319 BOERNE STAGE RD SAN ANTONIO, TX 78255",78255,"{'Latitude': 29.67749, 'Longitude': -98.661255}"
1,87260,"24424 BLANCO RD SAN ANTONIO, TX 78260",78260,"{'Latitude': 29.7011887, 'Longitude': -98.5204..."
2,94234,"2227 SAWGRASS RDG SAN ANTONIO, TX 78260",78260,{}
3,94265,"2243 WINDING VW SAN ANTONIO, TX 78260",78260,{}
4,94696,"1115 LINKS CV SAN ANTONIO, TX 78260",78260,"{'Latitude': 29.676438, 'Longitude': -98.47516..."


In [10]:
# Now grab the blank coordinates with zip codes
skip = 0
if skip == 0:
    y = mapping_crim_df[mapping_crim_df.coord == {}].property_zip.unique().tolist()
    zip_coords = []

    for item in y:
        d = {}
        a = geocode(item, exactly_one=True, timeout=60)
        try:
            d["Latitude"] = a.latitude
        except:
            pass
        try:
            d["Longitude"] = a.longitude
        except:
            pass
        zip_coords.append(d)
        
    zip_coords_dict = {}
    for i, j in enumerate(y):
        print(zip_coords[i],j)
        zip_coords_dict[j] = zip_coords[i]
        
    # Map onto coord column
    mask = mapping_crim_df[mapping_crim_df.coord=={}].index
    mapping_crim_df.loc[mask,'coord'] = mapping_crim_df.iloc[mask].property_zip.map(zip_coords_dict)

{'Latitude': 48.95928802921881, 'Longitude': 2.066597451511495} 78260
{'Latitude': 58.8200292378509, 'Longitude': 24.459501633992947} 78218
{'Latitude': 29.64659346855906, 'Longitude': -98.49094131651458} 78258
{'Latitude': 29.7942343, 'Longitude': -98.7528904} 78006


In [96]:
# save the original output
mapping_crim_df.to_csv('../data/mapping/mapping_crim_coord.csv',index=False)

In [75]:
# TOUTANT BEAUREGARD is giving a false geolocation; need to manually fix
# -98.698425 Y: 29.676485
mapping_crim_df2 = mapping_crim_df.copy()

coord_mask = mapping_crim_df2['crim_address'].str.startswith("TOUTANT BEAUREGARD")
mapping_crim_df2.loc[coord_mask,'coord'] = {'Latitude': 29.676485, 'Longitude':-98.698425}

In [87]:
# have to do the same for 10623 Star Mica; 29.682814, -98.698357
star_mica_mask = mapping_crim_df2['crim_address'].str.startswith("10623 STAR")
# mapping_crim_df2.loc[star_mica_mask,'coord'] = {'Latitude': 29.682814, 'Longitude': -98.698357}
mapping_crim_df2.loc[star_mica_mask,'coord']

45    {'Latitude': 29.7942343, 'Longitude': -98.7528...
Name: coord, dtype: object

In [92]:
# Pull out data for mapping
data = [[row['coord']['Latitude'],row['coord']['Longitude'],row['crim_address']] for index, row in mapping_crim_df2.iterrows()]

In [93]:
# Use folium to map each point
m = folium.Map(
    location=[29.583151, -98.51671949573658],
    zoom_start=10
)

for i in data:
    folium.Marker(
        i[:2],
        popup=i[2],
        icon=folium.Icon(color='red',icon='map-marker',prefix='fa')
    ).add_to(m)

In [94]:
folium.GeoJson(data='../data/mapping/Bexar_County.geojson').add_to(m)

In [95]:
display(m)

# Save as html
#     m.save("crim_prop_map.html")